In [2]:
import pandas as pd
from ggplot import *

project_id = "the-d-team-164512"

In [13]:
query ="""
    with p as (
    select  pat_state, count(distinct patient_id) as pat_dup_cnt
    from dthon.duplicates_enriched
    group by pat_state),
    p_l as (
    select count(distinct patient_id) as pat_cnt, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as pat_state2
    from dthon.patients_lookup
    group by pat_state2),
    s as (
    select count(distinct store_id) as str_dup_cnt, str_state
    from dthon.duplicates_enriched
    group by str_state),
    s_l as (
    select count(distinct store_id) as str_cnt, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as str_state2
    from dthon.stores_lookup
    group by str_state2)
    select p.*, p_l.pat_cnt, s.str_dup_cnt, s_l.str_cnt
    from p
    left join p_l
    on p.pat_state = p_l.pat_state2
    left join s
    on p.pat_state = s.str_state
    left join s_l
    on p.pat_state = s_l.str_state2
  """


First we would like to check whether all tasmanian patients have duplicates and all tasmanian stores also.

In [11]:
dup_pat_str = pd.read_gbq(query, project_id,dialect='standard')
dup_pat_str

Requesting query... ok.
Query running...
Query done.
Processed: 49.6 Mb

Retrieving results...
  Got page: 1; 100% done. Elapsed 10.96 s.
Got 7 rows.

Total time taken 10.97 s.
Finished at 2017-04-21 00:27:48.


,pat_state,pat_dup_cnt,pat_cnt,str_dup_cnt,str_cnt
0,3,602,180800,496,774
1,2,1588,201586,694,1085
2,5,155,36478,130,192
3,4,287,98990,358,487
4,6,82,20972,86,180
5,7,19078,19078,72,73
6,0,15,448,24,30


We see that one store doesn't generate duplicates (tasmania - 7). Which one it is and whether it has any transactions

In [14]:
query = """
select distinct store_id as str_cnt
from dthon.stores_lookup
where LPAD(LPAD(CAST(postcode as string),4,'0'), 1) = '7'
and store_id not in (select distinct store_id as str_dup_cnt
from dthon.duplicates_enriched
where str_state = '7')
"""
store_no_dup = pd.read_gbq(query, project_id,dialect='standard')
store_no_dup  

Requesting query... ok.
Query running...
Query done.
Processed: 20.5 Mb

Retrieving results...
Got 1 rows.

Total time taken 4.64 s.
Finished at 2017-04-21 00:35:49.


,str_cnt
0,1307


In [15]:
query = """
select Patient_ID
from dthon.transactions
where Store_ID = 1307
"""
store_1307 = pd.read_gbq(query, project_id,dialect='standard')
store_1307 

Requesting query... ok.
Query running...
Query done.
Processed: 907.1 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.88 s.
Finished at 2017-04-21 00:35:57.


,Patient_ID


That explains why there were no duplicates. On the other hand side:
* why is there a store without transactions
* how many are there

Before  that, let's check if all transactions from all tasmanian stores are duplicated.

In [16]:
query = """
with tr as (
select count(1) as cnt_all, LPAD(LPAD(CAST(postcode as string),4,'0'), 1) as state
from dthon.transactions tr
Left join dthon.stores_lookup as sl
on sl.Store_id = tr.Store_ID
group by state
), dup as (
select str_state, sum(cnt) as cnt_dup
from dthon.duplicates_enriched 
group by str_state
)
select dup.*, tr.cnt_all
from dup
left join tr
on tr.state = dup.str_state
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region

Requesting query... ok.
Query running...
Query done.
Processed: 474.1 Mb

Retrieving results...
Got 8 rows.

Total time taken 6.07 s.
Finished at 2017-04-21 00:48:58.


,str_state,cnt_dup,cnt_all
0,7,3785978,3835232.0
1,4,14263,9489847.0
2,0,2066,68213.0
3,3,17902,19325433.0
4,6,2181,1902308.0
5,None,60906,NaN
6,2,28368,21234352.0
7,5,4769,3532609.0


There are some stores with no state assigned:
* wrong query to create duplicates dataset?
* stores without postcode in lookup?
* stores in transactions without foreign code in lookup file?

Let's check how many stores have this problem.

In [17]:
query = """
select Store_ID, sum(cnt)
from dthon.duplicates_enriched
where str_state is null
group by Store_ID
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region

query = """
select *
from dthon.stores_lookup
where Store_ID = 408
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region


query = """
select *
from dthon.stores_lookup
where postcode is null
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region

Requesting query... ok.
Query running...
Query done.
Processed: 35.5 Mb

Retrieving results...
Got 0 rows.

Total time taken 3.03 s.
Finished at 2017-04-21 16:41:47.
Requesting query... ok.
Query running...
Query done.
Processed: 79.5 kb

Retrieving results...
Got 1 rows.

Total time taken 3.81 s.
Finished at 2017-04-21 16:41:53.
Requesting query... ok.
Query running...
Query done.
Processed: 79.5 kb

Retrieving results...
Got 0 rows.

Total time taken 2.49 s.
Finished at 2017-04-21 16:41:58.


,Store_ID,StateCode,postcode,IsBannerGroup


So we have found out that:
* Only 408 among duplicates shows this behaviour
* This is caused by null postcode (checked in the source file, also)
* 408 is the only store with null postcode

(You can check in source data, I updated big query tables) 

Let's update it and set the postcode to 7001 (no other store has this) in relevant tables.
Now let's see how are the duplicates other than from tasmanian patients distributed

In [4]:
query = """
with tr as (
select count(1) as cnt_all, LPAD(LPAD(CAST(sl.postcode as string),4,'0'), 1) as state
from dthon.transactions tr
Left join dthon.stores_lookup as sl
on sl.Store_id = tr.Store_ID
left join dthon.patients_lookup as pl
on pl.Patient_id = tr.Patient_ID
where LPAD(LPAD(CAST(pl.postcode as string),4,'0'), 1) != '7'
group by state
), dup as (
select str_state, sum(cnt) as cnt_dup
from dthon.duplicates_enriched 
where pat_state !='7'
group by str_state
)
select dup.*, tr.cnt_all
from dup
left join tr
on tr.state = dup.str_state
"""
dup_tr_region = pd.read_gbq(query, project_id,dialect='standard')
dup_tr_region

Requesting query... ok.
Query running...
Query done.
Processed: 941.9 Mb

Retrieving results...
  Got page: 1; 100% done. Elapsed 7.78 s.
Got 7 rows.

Total time taken 7.79 s.
Finished at 2017-04-23 15:17:05.


,str_state,cnt_dup,cnt_all
0,3,2270,19309801
1,5,797,3528637
2,2,9566,21215550
3,4,1175,9476759
4,6,303,1900430
5,7,2174,53313
6,0,18,66165


We can see that 50k transactions from tasmanian stores were not duplicated. Let's see in what stores they occurred

In [15]:
dup_tr_region.columns[1]

'cnt_dup'